In [ ]:
import numpy as np
import pandas as pd
import json

from pathlib import Path  


In [ ]:
data = []

In [ ]:
with open('./DATA/all_data.json') as d:
    data = json.load(d)
    
rows = []
subj = []
keywords = []
affiliations = []

for record in data:
    try:
        title = record['abstracts-retrieval-response']['coredata']['dc:title']
        publish_year = record['abstracts-retrieval-response']['item']['ait:process-info']['ait:date-sort']['@year']
        eid = record['abstracts-retrieval-response']['coredata']['eid']
        aggregation_type = record['abstracts-retrieval-response']['coredata']['prism:aggregationType']
        authors = record['abstracts-retrieval-response']['authors']['author']
        author_count = len(authors)
        if record['abstracts-retrieval-response']['language']:
            language = record['abstracts-retrieval-response']['language']['@xml:lang']
        else: language = None
        
        rows.append({
            'Title': title,
            'Publish_year': publish_year,
            'Eid': eid,
            'Aggregation_type': aggregation_type,
            'Author_Count': author_count,
            'Language' : language
        })
        
        areas = record['abstracts-retrieval-response']['subject-areas']['subject-area']
        for area in areas:
            subj.append({
                'Eid': eid,
                'Subject_Area': area["$"],
                'Subject_Code': area["@code"],
            })
            
        keyword = record['abstracts-retrieval-response']['item']['bibrecord']['head']['citation-info']['author-keywords']['author-keyword']
        for key in keyword:
            keywords.append({
                'Eid':eid,
                'Keyword':key["$"],
                'Language':key["@xml:lang"],
                'Original':key["@original"]
            })
            
        affils = record['abstracts-retrieval-response']['affiliation']
        for affil in affils:
            affiliations.append({
                'Eid': eid,
                'Affil_Id':affil["@id"],
                'Affil_Name':affil["affilname"],
                'Affil_Country':affil["affiliation-country"],
                'Affil_City':affil["affiliation-city"]
            })
        
    except:
        pass

df = pd.DataFrame(rows)

subject_df = pd.DataFrame(subj)

keyword_df = pd.DataFrame(keywords)

affil_df = pd.DataFrame(affiliations)

In [ ]:
df

In [ ]:
filepath = Path('DATA/main.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath, index=False) 

In [ ]:
subject_df

In [ ]:
keyword_df

In [ ]:
affil_df

In [ ]:
filepath = Path('DATA/subject.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
subject_df.to_csv(filepath, index=False) 

In [ ]:
filepath = Path('DATA/keyword.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
keyword_df.to_csv(filepath, index=False) 

In [ ]:
filepath = Path('DATA/affiliate.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
affil_df.to_csv(filepath) 

In [ ]:
subject_df = subject_df.drop_duplicates()

subject_df['Subject_Area'].value_counts()

In [ ]:
subject_df['Subject_Code'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:

unique_eid_df = df.drop_duplicates(subset='Eid')

unique_eid_df['Aggregation_type'].value_counts()

In [ ]:
merge_table = df.merge(subject_df,on='Eid',how='left')
merge_table